In [1]:
from model_lomba import *

model = switching_model(1,1,5,2)

#Di sini satuan waktunya dalam menit
num_of_steps = 24*60 # 1 day
for i in range(num_of_steps):
    model.step()

Ini create baterai
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Ini create motor
16
id baterai motorist
0
Ini create station
17
id baterai station
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
